## 1. Loading prereqs

In [ ]:
try:
    from google.colab import drive
    IN_COLAB = True
    print("Running on Google Colab. ")
except:
    IN_COLAB = False
    print("Not running on Google Colab. ")

In [ ]:
if IN_COLAB:
    !pip install facenet-pytorch  # fornisce modelli pre-addestrati PyTorch per compiti di riconoscimento facciale
    !pip install Pillow # aggiunge il supporto per l'apertura, la manipolazione e il salvataggio di molti diversi formati di file immagine.

### Connect to Drive

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

## 2. Load NN1

### Load pre-trained model





In [ ]:
# utilizzo la libreria facenet_pytorch per caricare il modello InceptionResnetV1 preaddestrato sul dataset VGGFace2 e abilitare la classificazione.
from facenet_pytorch import InceptionResnetV1, MTCNN
import torch


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print('Running on device: {}'.format(device))

resnet = InceptionResnetV1(pretrained='vggface2', device=device).eval()
resnet.classify = True
resnet = resnet.to(device)

#### Loading labels of model

In [ ]:
import numpy as np
import tensorflow as tf

# Il modello è addestrato sulle seguenti Labels:
# Carico le labels del dataset VGGFACE
fpath = tf.keras.utils.get_file('rcmalli_vggface_labels_v2.npy',
                             "https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_labels_v2.npy",
                             cache_subdir="./")
LABELS = np.load(fpath) # List of name

for i in range(len(LABELS)):
  LABELS[i] = LABELS[i].strip().replace(' ', '').replace('"', '')

## 2. Load NN2


#### Load repository for NN2

In [ ]:
# se non è presente la cartella VGGFACE2_pytorch  clone il repository
import os
if not os.path.exists('VGGFace2_pytorch'):
    !git clone https://github.com/cydonia999/VGGFace2-pytorch.git
    !mv VGGFace2-pytorch VGGFACE2_pytorch


### Import from repository and import operation

In [ ]:
%cd drive/Shareddrives/AI4CYBSEC/

In [ ]:
import torch
from VGGFace2_pytorch.models import senet as SENet
from VGGFace2_pytorch.models.resnet import resnet50 as ResNet
from VGGFace2_pytorch import utils
from VGGFace2_pytorch.trainer import Validator
from torch.utils.data import DataLoader
from VGGFace2_pytorch.datasets.vgg_face2 import VGG_Faces2
import os
from torch.nn.modules.loss import CrossEntropyLoss

### Load Model from pickel file

In [ ]:
model = SENet.senet50(num_classes = 8631, include_top = True)
weights_pickel = "./in_progress/senet50_ft_weight.pkl"
utils.load_state_dict(model, weights_pickel)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.to(device)
import torchsummary
torchsummary.summary(model, (3, 224, 224))

## 3. Load Test Set

#### Load the label of Test Set (for NN1)

In [ ]:
# set the path for the dataset
if IN_COLAB:
    path_dataset = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset"
else:
    path_dataset = "./face_dataset"
identity_meta_NN1_name = "meta_identity_NN1.csv"

import pandas as pd
import os

path_identity_csv =os.path.join(path_dataset,identity_meta_NN1_name)
identity_meta_NN1 = pd.read_csv(path_identity_csv)

## 4. Mapping different label encoding

### Mapping label for NN1

In [ ]:
# I want a dictonary related to the label of the Test Set that map the name of celebrities with label associated
name_to_id = {}
id_to_name = {}
for index, row in identity_meta_NN1.iterrows():
    # Ora puoi accedere ai valori di ogni riga come segue:
    class_id = row['Class_ID']
    name = row['Name']
    name_to_id[name]=class_id
    id_to_name[class_id] = name

### Mapping label for NN2

#### Function to create image list file for NN2 evaluation

In [ ]:
import os
def create_image_list_file(root_dir, output_file, ext = '.jpg'):

    image_paths = []

    for class_id in os.listdir(root_dir):
        class_dir = os.path.join(root_dir, class_id)

        if os.path.isdir(class_dir):

            for filename in os.listdir(class_dir):

                if filename.endswith(ext):
                    image_path = f"{os.path.basename(root_dir)}/{class_id}/{filename}"
                    image_paths.append(image_path)

    with open(output_file, 'w') as f:
        for image_path in image_paths:
            f.write(image_path + '\n')

    print(f"File di output creato con successo: {output_file}")

#### Mapping pipeline NN2

In [ ]:
test_set_NN2 = "test_set_MTCNN_NN2"

if IN_COLAB:
    output_file = '/content/drive/Shareddrives/AI4CYBSEC/image_list_file_NN2.txt'
    meta_file = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset/identity_meta.csv"
else:
    output_file = '/./image_list_file_NN2.txt'
    meta_file = "./face_dataset/identity_meta.csv"

root_dir = os.path.join(path_dataset,test_set_NN2)
create_image_list_file(root_dir, output_file)
id_label_dict = utils.get_id_label_map(meta_file)

## 5. Dataset Class

In [ ]:
from PIL import Image
import os
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

class VGGFace2Dataset(Dataset):
    def __init__(self, root_dir, image_size=(160, 160), transform=None):
        self.root_dir = root_dir
        self.image_size = image_size
        self.transform = transform

        # List of files in the dataset
        self.file_list = []
        for root, dirs, files in os.walk(self.root_dir):
            for file in files:
                self.file_list.append(os.path.join(root, file))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path).resize(self.image_size)

        # Extract the label from the file path
        label = os.path.split(os.path.dirname(img_path))[-1]

        if self.transform:
            img = self.transform(img)

        return img, label

## 6. Evaluate model NN1


### Utility Functions for NN1 with mapping labels

In [ ]:
from PIL import Image
from torchvision import transforms
import torch
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm


def make_inference(model, image_tensors, name_to_id, device):
    """
    Takes input image tensor and returns the label associated with the network's prediction.

    """
    model.to(device)
    model.eval()
    # Move image tensors to the specified device
    image_tensors = image_tensors.to(device)

    probs = model(image_tensors)
    #print("probs", probs)

    # Get the number of elements along the first dimension
    num_elements = probs.size(0)

    # Initialize two lists to store the argmax
    argmax_list_1 = []
    argmax_list_2 = []

    # Compute argmax for each element along the first dimension
    for i in range(num_elements):
        #target_class = np.array(probs[i].detach().cpu().numpy()).argmax()  # Move to CPU for numpy operations
        target_class = torch.argmax(probs[i]).item()
        argmax_list_1.append(name_to_id[LABELS[target_class]])
        argmax_list_2.append(target_class)

    return argmax_list_1, argmax_list_2


def validate(dataset, model, name_to_id, device):
    """
    Validates a model on a dataset and returns the accuracy.

    Args:
        dataset: Dataloader to validate the model on.
        model: Model to validate.
        device: Device to perform inference on.

    Returns:
        accuracy: Accuracy of the model on the dataset.
    """
    model.eval()
    correct_predictions = 0
    total_samples = len(dataset) * dataset.batch_size

    with torch.no_grad():  # Disable gradient calculation
        for images, labels in tqdm(dataset, desc="Validating model"):
            #images = mtcnn(images)
            predicted_classes, _ = make_inference(model, images, name_to_id, device)
            correct_predictions += sum(pred == label for pred, label in zip(predicted_classes, labels))

    # Compute accuracy
    accuracy = correct_predictions / total_samples
    return accuracy


def plot_image(image, label):
  """
  prende in ingresso le PIL.Image del campione originale e del corrispondete adversarial sample e li plotta
  """
  plt.figure()
  plt.matshow(image)
  plt.title("Model Prediction: {}".format(label))
  plt.show()

### Validation on Clean Data

In [ ]:
# Create transform for image resizing and normalization
data_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

if IN_COLAB:
    test_set_path = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset/test_set_MTCNN"
else:

    test_set_path = "./face_dataset/test_set_MTCNN"
# Define dataset
dataset = VGGFace2Dataset(root_dir=test_set_path, transform=data_transform)
dataset_len = len(dataset)

# Check the length of the dataset
print("Dataset length:", dataset_len)

# Create DataLoader
batch_size = 1
dataloader_NN1 = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# accuracy_clean_data = validate(dataloader, resnet, name_to_id, device)
accuracy_clean_data = 0.832
print("\nAccuracy on clean data: " + str(round(accuracy_clean_data, 3)))

## 7. Evaluate Model NN2

### Utility functions for NN2


In [ ]:
def preprocessing_on_tensor(img_tensor, mean_bgr=np.array([91.4953, 103.8827, 131.0912])):
    """
    Perform preprocessing on the input image tensor for the model.

    :param img_tensor: immagine with shape (C, H, W) and values in [0, 1]
    :return: immagine normalizzata using mean_bgr with shape (1, C, H, W)

    """
    img = img_tensor.squeeze(0)
    img = img.numpy()
    img = (img * 255).astype(np.uint8)
    mean_bgr = np.array([91.4953,103.8827, 131.0912])
    # img è C x H x W --> H x W x C
    img = np.transpose(img, (1, 2, 0))
    img = img[:, :, ::-1]  # RGB -> BGR
    img = img.astype(np.float32)
    img -= mean_bgr
    img = img.transpose(2, 0, 1)
    img_tensor = torch.from_numpy(img).unsqueeze(0)
    return img_tensor


def make_inference_NN2(model, img_tensor, device, with_preprocessing=True):
    """
    Esegue l'inferenza su un'immagine.
    :param model: modello
    :param img_tensor: immagine trasformata
    :param device: dispositivo
    :return: predizione
    """
    if with_preprocessing:
        img_tensor = img_tensor.squeeze(0)
        img_tensor = preprocessing_on_tensor(img_tensor)
    model.eval()
    img_tensor = img_tensor.to(device)

    with torch.no_grad():
        output = model(img_tensor)
        pred = torch.argmax(output, dim=1).item()
    return pred

## 8. FGSM attack

In [ ]:
if IN_COLAB:
    !pip install adversarial-robustness-toolbox[all]
import art

### Utility Functions to perform attacks

In [ ]:
### Utility Functions to perform attacks
import numpy as np
import csv
from datetime import datetime
import json
import matplotlib
import matplotlib.pyplot as plt


def compute_perturbation(original_images, adversarial_samples):
    perturbations = []
    for original_image, adversarial_sample in zip(original_images, adversarial_samples):
        perturbation = np.mean(np.abs((np.array(original_image) - np.array(adversarial_sample))))
        perturbations.append(perturbation)
    return round(float(np.mean(perturbations)),3)


def sec_curve(strength_values, accuracies_values, constant_values, strength_name, target_class=None,
              attack=None, avg_perturbations=None, accuracy_on_target_class=None, network = "NN1"):
    fig, ax = plt.subplots()
    # Costruisci la stringa per i valori costanti
    constant_str = ', '.join([f'{key}: {value}' for key, value in constant_values.items()])
    line = ax.plot(np.array(strength_values), np.array(accuracies_values), 'b--', label=f'{network} - {constant_str}')


    # Aggiungi i valori costanti come parte della legenda
    if target_class:
        plt.title('Security Curve for Target Class {}'.format(target_class))
    else:
        plt.title('Security Curve')

    # Aggiungi l'attacco al titolo
    if attack:
        plt.title(f'{attack} - {plt.gca().get_title()}')

    plt.xlabel('Attack strength ({})'.format(strength_name))
    plt.ylabel('Accuracy Test')
    plt.grid()

    # Aggiungi il diagramma a barre di colore arancione per avg_perturbations
    if avg_perturbations:
        x = np.array(strength_values)
        ax2 = ax.twinx()
        bar = ax2.bar(x, avg_perturbations, color='orange', alpha=0.5, width=0.01, label='Avg Perturbations')
        ax2.set_ylabel('Avg Perturbations')



    if accuracy_on_target_class:
        # Aggiungi la curva di accuratezza per la classe target
        ax.plot(np.array(strength_values), np.array(accuracy_on_target_class), 'r--', label=f'{network} - mis_targ/miss')

    # Unisci le linee e le barre in una lista per la legenda
    handles, labels = ax.get_legend_handles_labels()
    if avg_perturbations:
        handles2, labels2 = ax2.get_legend_handles_labels()
        handles += handles2
        labels += labels2

    # Mostra la legenda
    #plt.legend(handles, labels, loc='upper right', shadow=True, fontsize='small')
    #voglio che la legenda sia fuori dal grafico
    plt.legend(handles, labels, loc='upper left', bbox_to_anchor=(1.2, 1), shadow=True, fontsize='small')
    plt.show()


def save_to_csv(attack_name, targeted, target_class, strength_name, strength_values, accuracy_values, constant_values, avg_perturbations, file_path, accuracy_on_target_class=None):
    # Intestazione del file CSV
    header = ["timestamp", "attacco", "targeted", "target_class", "strength_name", "strength_values", "accuracy_values", "constant_values", "avg_perturbations", "accuracy_on_target_class"]

    # Creazione della tupla con i valori da scrivere nel CSV
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # Converti le liste di valori in stringhe JSON
    strength_values_json = json.dumps(strength_values)
    accuracy_values_json = json.dumps(accuracy_values)
    avg_perturbations_json = json.dumps(avg_perturbations)  # Converti avg_perturbations in una stringa JSON
    # Modifica: formatta strength_values come una lista di numeri invece di una stringa JSON
    accuracy_values_list = [float(val) for val in accuracy_values]
    # Modifica: converti accuracy_values in una lista di numeri
    row = (timestamp, attack_name, targeted, target_class, strength_name, strength_values_json, accuracy_values_list, json.dumps(constant_values), avg_perturbations_json)

    # Se accuracy_on_target_class è fornito e non è None, includilo nella tupla
    if accuracy_on_target_class is not None:
        accuracy_on_target_class_json = json.dumps(accuracy_on_target_class)  # Converti accuracy_on_target_class in una stringa JSON
        row += (accuracy_on_target_class_json,)
    else:
        row += (None,)  # Aggiungi None alla tupla

    # Scrittura nel file CSV in modalità append ('a')
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if file.tell() == 0:  # Se il file è vuoto, scrivi l'intestazione
            writer.writerow(header)
        writer.writerow(row)


def read_csv_and_plot(csv_file_path, network="NN1"):
    with open(csv_file_path, mode='r') as file:
        reader = csv.DictReader(file)

        for row in reader:
            timestamp = row["timestamp"]
            attack_name = row["attacco"]
            targeted = row["targeted"]
            target_class = row["target_class"]
            strength_name = row["strength_name"]
            strength_values = json.loads(row["strength_values"])
            accuracy_values = json.loads(row["accuracy_values"])
            constant_values = json.loads(row["constant_values"])
            avg_perturbations = json.loads(row["avg_perturbations"])
            accuracy_on_target_class = json.loads(row["accuracy_on_target_class"]) if row["accuracy_on_target_class"] else None

            # Controlla se accuracy_values è una stringa JSON e la elabora correttamente
            if isinstance(accuracy_values, str):
                accuracy_values = json.loads(accuracy_values)

            # Controlla se strength_values è una stringa JSON e la elabora correttamente
            if isinstance(strength_values, str):
                strength_values = json.loads(strength_values)

            # Controlla se strength_values è una stringa JSON e la elabora correttamente
            if isinstance(avg_perturbations, str):
                avg_perturbations = json.loads(avg_perturbations)

            # Chiama la funzione sec_curve per plottare la curva
            sec_curve(strength_values, accuracy_values, constant_values, strength_name, target_class, attack_name, avg_perturbations, accuracy_on_target_class, network)

### Initialization of FGSM attack

### Prepare dataset for attacks

In [ ]:
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod

from torch import nn, optim
import torchvision.transforms as transforms

import numpy as np

input_shape = (3,160,160)
nb_classes = 8631
loss = nn.CrossEntropyLoss()    # Triplett loss = nn.TripletMarginLoss()
optimizer = optim.Adam(resnet.parameters())
classifier = PyTorchClassifier(model=resnet, loss=loss, input_shape=input_shape, nb_classes=nb_classes, optimizer=optimizer, clip_values=(0, 1))

attack_name = "FGSM"
results_csv = "./attack_results.csv"
PERFORM_ATTACK_NN1 = False
network = 'NN1'

In [ ]:
if PERFORM_ATTACK_NN1:
  images_list = []
  labels_list = []
  for image, label in tqdm(dataloader_NN1, desc="Preparing dataset"):
      images_list.append(image.numpy())
      labels_list.append(label)

### Perform untargeted FGSM attacks

In [ ]:
if PERFORM_ATTACK_NN1:
  from tqdm import tqdm

  epsilon = 0.1
  targeted = False
  fgsm_untargeted_attacker = FastGradientMethod(estimator=classifier, eps=epsilon, targeted=targeted)

  eps_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
  accuracy_values_untargeted = [accuracy_clean_data]
  avg_perturbations = [0]

  for eps in tqdm(eps_range, desc='Generating {} attacks'.format(attack_name)):
      fgsm_untargeted_attacker.set_params(**{'eps': eps})
      nb_correct_eps = 0
      images_adv_list = []
      for img, label in zip(images_list, labels_list):
          x_test_adv = fgsm_untargeted_attacker.generate(img)
          images_adv_list.append(x_test_adv)
          x_test_adv = torch.tensor(x_test_adv)
          x_test_adv_pred,_ = make_inference(resnet, x_test_adv, name_to_id, device)
          if x_test_adv_pred[0] == label[0]:
              nb_correct_eps += 1
          break
      accuracy_values_untargeted.append(nb_correct_eps/dataset_len)
      avg_perturbations.append(compute_perturbation(images_list, images_adv_list))

  eps_range = [0] + eps_range
  constant_values = {}
  strength_name='eps'
  target_class = None
  accuracy_on_target_class = None

  sec_curve(eps_range, accuracy_values_untargeted, constant_values, strength_name, target_class, attack_name, avg_perturbations, accuracy_on_target_class, network)

  # save results on csv
  save_to_csv(attack_name, targeted, target_class, strength_name, eps_range, accuracy_values_untargeted, constant_values, avg_perturbations, results_csv)

### Perform targeted FGSM attacks

In [ ]:
if PERFORM_ATTACK_NN1:
  from tqdm import tqdm

  epsilon = 0.1
  targeted=True
  fgsm_targeted_attacker = FastGradientMethod(estimator=classifier, eps=epsilon, targeted=targeted)
  target_class = 0

  eps_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
  accuracy_values_targeted = [accuracy_clean_data]
  accuracy_on_target_class = [0]
  avg_perturbations = [0]

  targeted_label = np.array(target_class)
  one_hot_bob_targeted_label = np.expand_dims(tf.keras.utils.to_categorical(targeted_label, num_classes=nb_classes), axis=0)

  for eps in tqdm(eps_range, desc='Generating {} attacks'.format(attack_name)):
      fgsm_targeted_attacker.set_params(**{'eps': eps})
      nb_correct_eps = 0
      nb_target_eps = 0
      nb_misclassifications = 0
      images_adv_list = []
      for img, label in zip(images_list, labels_list):
          x_test_adv = fgsm_targeted_attacker.generate(img, one_hot_bob_targeted_label)
          images_adv_list.append(x_test_adv)
          x_test_adv = torch.tensor(x_test_adv)
          x_test_adv_pred,x_test_adv_pred_model = make_inference(resnet, x_test_adv, name_to_id, device)
          if x_test_adv_pred[0] == label[0]:
              nb_correct_eps += 1
          else:
            nb_misclassifications += 1
            if x_test_adv_pred_model[0] == target_class:
              nb_target_eps += 1

      accuracy_values_targeted.append(nb_correct_eps/dataset_len)
      accuracy_on_target_class.append(round(nb_target_eps/nb_misclassifications,3))
      avg_perturbations.append(compute_perturbation(images_list, images_adv_list))

  eps_range = [0] + eps_range
  constant_values = {}
  strength_name='eps'
  sec_curve(eps_range, accuracy_values_targeted, constant_values, strength_name, target_class, attack_name, avg_perturbations, accuracy_on_target_class, network)

  # save results on csv
  save_to_csv(attack_name, targeted, target_class, strength_name, eps_range, accuracy_values_targeted, constant_values, avg_perturbations, results_csv, accuracy_on_target_class)

In [ ]:
if IN_COLAB:
    results_csv = "/content/drive/Shareddrives/AI4CYBSEC/results/FGSM/attack_results_NN1.csv"
else:
    results_csv = "./results/FGSM/attack_results_NN1.csv"
network = 'NN1'
read_csv_and_plot(results_csv, network)

# Test Transferibility

In [ ]:
PERFORM_ATTACK_NN2 = False

### Load list of images and labels

In [ ]:

data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

if IN_COLAB:
  test_set_path = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset/test_set_MTCNN_NN2"
else:
  test_set_path = "./face_dataset/test_set_MTCNN_NN2"
# Define dataset
dataset = VGGFace2Dataset(root_dir=test_set_path, image_size=(224,224), transform=data_transform)

# Check the length of the dataset
print("Dataset length:", len(dataset))

batch_size=1
dataloader_NN2 = DataLoader(dataset, batch_size=batch_size, shuffle=False)

images_list = []
labels_list = []
images_adv_list = []
for image, label in dataloader_NN2:
    image_numpy = image.numpy()
    images_list.append(image_numpy)
    labels_list.append(label)

In [ ]:
# dobbiamo fare la trasformazione che fa SENET
correct = 0
for  img, label in tqdm(zip(images_list, labels_list), total=len(images_list), desc=" Validation on clean data with make_inference_NN2 RESNET LOADER"):
    img = torch.tensor(img)

    x_test_adv_pred = make_inference_NN2(model, img, device, with_preprocessing=True)
    if x_test_adv_pred == id_label_dict[label[0]]:
      correct +=1
accuracy_clean_data_NN2 = correct/len(images_list)
print("Accuracy on clean data: ", accuracy_clean_data_NN2)


### Set variable Perform Attack

In [ ]:
if IN_COLAB:
    results_csv = "/content/drive/Shareddrives/AI4CYBSEC/results/FGSM/attack_results_NN2.csv"
else:
    results_csv = "./results/FGSM/attack_results_NN2.csv"

network = "NN2"

### Perform attack Error Generic Gray Box

In [ ]:
if PERFORM_ATTACK_NN2:
  from tqdm import tqdm

  epsilon = 0.1
  targeted = False
  fgsm_untargeted_attacker = FastGradientMethod(estimator=classifier, eps=epsilon, targeted=targeted)

  eps_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
  accuracy_values_untargeted = [accuracy_clean_data_NN2]
  avg_perturbations = [0]

  for eps in tqdm(eps_range, desc='Generating {} attacks'.format(attack_name)):
      fgsm_untargeted_attacker.set_params(**{'eps': eps})
      nb_correct_eps = 0
      images_adv_list = []
      for img, label in zip(images_list, labels_list):
          x_test_adv = fgsm_untargeted_attacker.generate(img)
          images_adv_list.append(x_test_adv)
          x_test_adv = torch.tensor(x_test_adv)
          x_test_adv_pred = make_inference_NN2(model, x_test_adv, device, with_preprocessing=True)

          if x_test_adv_pred == id_label_dict[label[0]]:
            nb_correct_eps +=1
      accuracy_values_untargeted.append(nb_correct_eps/dataset_len)
      avg_perturbations.append(compute_perturbation(images_list, images_adv_list))

  eps_range = [0] + eps_range
  constant_values = {}
  strength_name='eps'
  target_class = None
  accuracy_on_target_class = None

  sec_curve(eps_range, accuracy_values_untargeted, constant_values, strength_name, target_class, attack_name, avg_perturbations, accuracy_on_target_class, network)

  # save results on csv
  save_to_csv(attack_name, targeted, target_class, strength_name, eps_range, accuracy_values_untargeted, constant_values, avg_perturbations, results_csv)

### Perform attack Error Specific Gray Box

In [ ]:
if PERFORM_ATTACK_NN2:
  from tqdm import tqdm

  epsilon = 0.1
  targeted=True
  fgsm_targeted_attacker = FastGradientMethod(estimator=classifier, eps=epsilon, targeted=targeted)
  target_class = 0

  eps_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
  accuracy_values_targeted = [accuracy_clean_data_NN2]
  accuracy_on_target_class = [0]
  avg_perturbations = [0]

  targeted_label = np.array(target_class)
  one_hot_bob_targeted_label = np.expand_dims(tf.keras.utils.to_categorical(targeted_label, num_classes=nb_classes), axis=0)

  for eps in tqdm(eps_range, desc='Generating {} attacks'.format(attack_name)):
      fgsm_targeted_attacker.set_params(**{'eps': eps})
      nb_correct_eps = 0
      nb_target_eps = 0
      nb_misclassifications = 0
      images_adv_list = []
      for img, label in zip(images_list, labels_list):
          x_test_adv = fgsm_targeted_attacker.generate(img, one_hot_bob_targeted_label)
          images_adv_list.append(x_test_adv)
          x_test_adv = torch.tensor(x_test_adv)
          x_test_adv_pred = make_inference_NN2(model, x_test_adv, device, with_preprocessing=True)
          if x_test_adv_pred == id_label_dict[label[0]]:
            nb_correct_eps +=1
          else:
            nb_misclassifications += 1
            if x_test_adv_pred == target_class:
              nb_target_eps += 1

      accuracy_values_targeted.append(nb_correct_eps/dataset_len)
      accuracy_on_target_class.append(round(nb_target_eps/nb_misclassifications,3))
      avg_perturbations.append(compute_perturbation(images_list, images_adv_list))

  eps_range = [0] + eps_range
  constant_values = {}
  strength_name='eps'
  sec_curve(eps_range, accuracy_values_targeted, constant_values, strength_name, target_class, attack_name, avg_perturbations, accuracy_on_target_class, network)

  # save results on csv
  save_to_csv(attack_name, targeted, target_class, strength_name, eps_range, accuracy_values_targeted, constant_values, avg_perturbations, results_csv, accuracy_on_target_class)

In [ ]:
if IN_COLAB:
    results_csv = "/content/drive/Shareddrives/AI4CYBSEC/results/FGSM/attack_results_NN2.csv"
else:
    results_csv = "./results/FGSM/attack_results_NN2.csv"

network = "NN2"
read_csv_and_plot(results_csv, network)

### FGSM NN1 with Defense

#### Load Robus Detector for pre-processing

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if IN_COLAB:
    directory = "/content/drive/Shareddrives/AI4CYBSEC/models"
else:
    directory = ".\models"
def load_model(model, model_path):
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Definisci il modello mobilenet_v2
model = models.mobilenet_v2(pretrained=True)

# Sostituisci il classificatore dell'ultimo layer con un nuovo classificatore
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)

model = model.to(device)


# carica i pesi del modello addestrato
defence = load_model(model, os.path.join(directory,'mobilenetv2_best_binary_classifier.pth'))

import torchsummary

# Stampa un riassunto del modello
torchsummary.summary(defence, (3, 160, 160))

#### Utility for inferece

In [ ]:
def make_inference_defence(model, img_tensor, device):
    #img must be a tensor with shape (N, C, H, W)
    model.eval()
    img_tensor = img_tensor.to(device)
    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)

    return predicted.item()

def make_inference_NN1_with_defense(model,img_tensor, name_to_id, defense_model, device, isClean):
    model.to(device)
    defense_model.to(device)
    prediction_defense = make_inference_defence(defense_model, img_tensor, device)
    if prediction_defense == 1:
        if not isClean:
            return 1, None
        return 0 ,None

    return make_inference(model, img_tensor, name_to_id, device)

def validate_with_defence(dataloader, model, name_to_id, device, defence_model, clean_data = True):
    """
    Validates a model on a dataset and returns the accuracy.

    Args:
        dataset: Dataloader to validate the model on.
        model: Model to validate.
        device: Device to perform inference on.

    Returns:
        accuracy: Accuracy of the model on the dataset.
    """
    model.to(device)
    model.eval()
    correct_predictions = 0
    total_samples = len(dataloader) * dataloader.batch_size
    num_skipped_samples = 0
    with torch.no_grad():  # Disable gradient calculation
        for images, labels in tqdm(dataloader, desc="Validating model"):
            predicted_classes, _= make_inference_NN1_with_defense(model, images, name_to_id, defence_model, device, clean_data)
            if predicted_classes == 1:# significa che ho predetto come adv un campione  adv
                num_skipped_samples += 1
                correct_predictions += 1
                continue
            elif predicted_classes == 0:# significa che ho predetto come adv un campione  clean
                num_skipped_samples += 1
                continue
            # DATO CHE C'è il continue è come se avessi un else:
            correct_predictions += sum(pred == label for pred, label in zip(predicted_classes, labels))


    # Compute accuracy
    accuracy = correct_predictions / total_samples
    return accuracy, num_skipped_samples

#### Validation on clean data with defense

In [ ]:
# Create transform for image resizing and normalization

data_transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

if IN_COLAB:
  test_set_path = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset/test_set_MTCNN"
else:
  test_set_path = "./face_dataset/test_set_MTCNN"
# Define dataset
dataset = VGGFace2Dataset(root_dir=test_set_path, transform=data_transform)

# Check the length of the dataset
print("Dataset length:", len(dataset))

# Create DataLoader
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
acc = validate(dataloader, resnet, name_to_id, device)
print("\n Accuracy without defense "+str(acc))

acc_with_defence, num_skipped_samples = validate_with_defence(dataloader, resnet, name_to_id, device, defence, clean_data = True)
print("\n Accuracy with defense "+str(acc_with_defence))


### FGSM attack with defence -- NN1

In [ ]:
PERFORM_ATTACK_NN1_with_defense= False

#### Load list of images and labels

In [ ]:
images_list = []
labels_list = []
images_adv_list = []
for image, label in dataloader:
    # Effettua le predizioni del modello
    # image numpy on device
    image_numpy = image.numpy()
    images_list.append(image_numpy)
    labels_list.append(label)

#### Set variable Perform Attack

In [ ]:
if IN_COLAB:
    results_csv = "/content/drive/Shareddrives/AI4CYBSEC/results/FGSM/attack_results_NN1_with_defense.csv"
else:
    results_csv = "./results/FGSM/attack_results_NN1_with_defense.csv"
network = "NN1_with_defense"
read_csv_and_plot(results_csv, network)

#### Perform attack Error Generic Gray Box

In [ ]:
if PERFORM_ATTACK_NN1_with_defense:
  from tqdm import tqdm
  from art.attacks.evasion import FastGradientMethod
  import time

  from tqdm import tqdm

  epsilon = 0.1
  targeted = False
  fgsm_untargeted_attacker = FastGradientMethod(estimator=classifier, eps=epsilon, targeted=targeted)

  eps_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
  accuracy_values_untargeted = [acc_with_defence]
  avg_perturbations = [0]

  for eps in tqdm(eps_range, desc='Generating {} attacks'.format(attack_name)):
      fgsm_untargeted_attacker.set_params(**{'eps': eps})
      num_skipped_samples = 0
      nb_correct_eps = 0
      images_adv_list = []
      for img, label in zip(images_list, labels_list):
        x_test_adv = fgsm_untargeted_attacker.generate(img)
        images_adv_list.append(x_test_adv)
        x_test_adv = torch.tensor(x_test_adv)
        x_test_adv_pred, _= make_inference_NN1_with_defense(resnet, x_test_adv, name_to_id, defence, device, isClean=False)

        if x_test_adv_pred == 1:# significa che ho predetto come adv un campione  adv
          num_skipped_samples += 1
          nb_correct_eps += 1
          continue
        elif x_test_adv_pred == 0:# significa che ho predetto come adv un campione  clean
            num_skipped_samples += 1
            continue
        if x_test_adv_pred[0] == label[0]:
          nb_correct_eps +=1
      accuracy_values_untargeted.append(nb_correct_eps/len(images_list))
      avg_perturbations.append(compute_perturbation(images_list, images_adv_list))
      print("accuracy: ", nb_correct_eps/len(images_list))
      print("Number of skipped samples: ", num_skipped_samples)

  eps_range = [0] + eps_range
  constant_values = {}
  strength_name='eps'
  target_class = None
  accuracy_on_target_class = None

  sec_curve(eps_range, accuracy_values_untargeted, constant_values, strength_name, target_class, attack_name, avg_perturbations, accuracy_on_target_class, network)

  # save results on csv
  save_to_csv(attack_name, targeted, target_class, strength_name, eps_range, accuracy_values_untargeted, constant_values, avg_perturbations, results_csv)

#### Perform attack Error Specific Gray Box

In [ ]:
if PERFORM_ATTACK_NN1_with_defense:
  from tqdm import tqdm
  from art.attacks.evasion import FastGradientMethod
  import time

  from tqdm import tqdm

  epsilon = 0.1
  targeted=True
  fgsm_targeted_attacker = FastGradientMethod(estimator=classifier, eps=epsilon, targeted=targeted)
  target_class = 0

  eps_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
  accuracy_values_untargeted = [acc_with_defence]
  avg_perturbations = [0]

  targeted_label = np.array(target_class)
  one_hot_bob_targeted_label = np.expand_dims(tf.keras.utils.to_categorical(targeted_label, num_classes=nb_classes), axis=0)

  for eps in tqdm(eps_range, desc=f'Generating {attack_name}'):
      fgsm_targeted_attacker.set_params(**{'eps': eps, 'eps_step':eps/10})
      num_skipped_samples = 0
      nb_correct_eps = 0
      error_equl_target=0
      images_adv_list = []
      for img, label in zip(images_list, labels_list):
          x_test_adv = fgsm_targeted_attacker.generate(img, one_hot_bob_targeted_label)
          images_adv_list.append(x_test_adv)
          x_test_adv = torch.tensor(x_test_adv)
          x_test_adv_pred, x_test_adv_pred_model = make_inference_NN1_with_defense(resnet, x_test_adv, name_to_id, defence, device, isClean=False)
          if x_test_adv_pred == 1:# significa che ho predetto come adv un campione  adv
            num_skipped_samples += 1
            nb_correct_eps += 1
            continue
          elif x_test_adv_pred == 0:# significa che ho predetto come adv un campione clean
              num_skipped_samples += 1
              continue
          if x_test_adv_pred[0] == label[0]:
            nb_correct_eps +=1
          elif x_test_adv_pred_model[0] == target_class:
            error_equl_target+=1
      accuracy_values_targeted.append(round(nb_correct_eps/dataset_len,3))
      avg_perturbations.append(compute_perturbation(images_list, images_adv_list))

      if len(images_list)-nb_correct_eps == 0:
          res = 0
      else:
          res = round(error_equl_target/(len(images_list)-nb_correct_eps),3)
      accuracy_on_target_class.append(res)
      print("accuracy: ", nb_correct_eps/len(images_list))
      print(f"avg_perturbations: {avg_perturbations}")
      print("Number of skipped samples: ", num_skipped_samples)
      print(f"accuracy_on_target_class: {accuracy_on_target_class}")

  eps_range = [0] + eps_range
  constant_values = {}
  strength_name='eps'
  sec_curve(eps_range, accuracy_values_targeted, constant_values, strength_name, target_class, attack_name, avg_perturbations, accuracy_on_target_class, network)

  # save results on csv
  save_to_csv(attack_name, targeted, target_class, strength_name, eps_range, accuracy_values_targeted, constant_values, avg_perturbations, results_csv, accuracy_on_target_class)

In [ ]:
if IN_COLAB:
    results_csv = "/content/drive/Shareddrives/AI4CYBSEC/results/FGSM/attack_results_NN1_with_defense.csv"
else:
    results_csv = "./results/FGSM/attack_results_NN1_with_defense.csv"
network = "NN1_with_defense"
read_csv_and_plot(results_csv, network)